In [1]:
# Desinstalar tudo que pode estar corrompido
!pip uninstall -y numpy sentence-transformers transformers huggingface-hub

# Reinstalar numpy primeiro (versão estável compatível com tudo)
!pip install -q numpy==1.24.4

# Reinstalar versões compatíveis de transformers e huggingface-hub
!pip install -q huggingface_hub==0.20.3 transformers==4.38.2

# Reinstalar sentence-transformers e outras bibliotecas do projeto
!pip install -q sentence-transformers spacy faiss-cpu pandas torch nltk

# Baixar modelo do spaCy e dados do WordNet
!python -m spacy download en_core_web_sm

import nltk
nltk.download("wordnet")

Found existing installation: numpy 2.3.0
Uninstalling numpy-2.3.0:
  Successfully uninstalled numpy-2.3.0
Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: huggingface-hub 0.33.0
Uninstalling huggingface-hub-0.33.0:
  Successfully uninstalled huggingface-hub-0.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtune 0.6.1 requires huggingface_hub[hf_transfer], which is not installed.
diffusers 0.33.1 requires huggingface-hub>=0.27.0, which is not installed.
peft 0.15.2 requires huggingface_hub>=0.25.0, which is not installed.
peft 0.15.2 requires transformers, which is not installed.
accelerate 1.

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# A primeira célula carrega e prepara os dados e cria um sistema de busca vetorial inteligente
#(pronto para expandir consultas e responder de forma mais precisa depois).

import spacy
import faiss
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
from nltk.corpus import wordnet
from datetime import datetime

# Baixar os dados necessários para WordNet
nltk.download("wordnet")

# Carregar modelos
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
t5_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Carregar dataset
df = pd.read_csv("amazon_with_sales.csv")

# Definir feriados e datas especiais
data_especial = {
    'New ano': '01-01',
    'Valentines dia': '06-12',
    'Mothers dia': '05-12',
    'Black Fridia': '11-24',
    'Christmas': '12-25'
}

# Função para enriquecer dados de data
def novo_data_info(data_str):
    data = datetime.strptime(data_str, "%Y-%m-%d")
    mes = data.month
    dia = data.day
    ano = data.year

    # Estações do ano (hemisfério norte)
    if mes in [12, 1, 2]:
        estacao = 'Winter'
    elif mes in [3, 4, 5]:
        estacao = 'Spring'
    elif mes in [6, 7, 8]:
        estacao = 'Summer'
    else:
        estacao = 'Autumn'

    # Verificar se a data coincide com datas especiais
    periodo_especial = 'None'
    for evento, evento_data in data_especial.items():
        evento_mes, evento_dia = map(int, evento_data.split('-'))
        if mes == evento_mes and dia == evento_dia:
            periodo_especial = evento

    return {
        'ano': ano,
        'mes': mes,
        'dia': dia,
        'estacao': estacao,
        'periodo_especial': periodo_especial
    }

# Aplicar enriquecimento de data
data_info = df['date'].apply(novo_data_info).apply(pd.Series)
df = pd.concat([df, data_info], axis=1)

# Criar dicionário de sinônimos para categorias
categorias = set(df["category"].dropna().str.lower().str.strip().unique())
categorias_sinonimos = {}
for category in categorias:
    terms = category.split("|")
    synonyms_set = set(terms)
    for term in terms:
        doc = nlp(term.replace("&", " "))
        for token in doc:
            for synset in wordnet.synsets(token.text):
                for lemma in synset.lemmas():
                    synonyms_set.add(lemma.name().replace("_", " "))
    categorias_sinonimos[category] = list(synonyms_set)

# Adicionar sinônimos ao texto do produto antes de gerar os embeddings
def expandir_descricao_do_produto(row):
    category = row["category"].lower().strip()
    synonyms = categorias_sinonimos.get(category, [])
    expanded_text = row["product_name"] + " " + row["about_product"] + " " + " ".join(categorias_sinonimos.get(category, []))
    return embedding_model.encode(expanded_text, normalize_embeddings=True)

# Criar embeddings vetoriais incluindo sinônimos
df["embedding"] = df.apply(expandir_descricao_do_produto, axis=1)

# Criar índice FAISS
dimension = df["embedding"].iloc[0].shape[0]
quantizer = faiss.IndexFlatL2(dimension)  # base para clusters
nlist = 100  # número de clusters (pode ajustar para datasets maiores ou menores)

index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)
embeddings_matrix = np.vstack(df["embedding"].values).astype("float32")
# Treinar o índice antes de adicionar (obrigatório para IVFFlat)
index.train(embeddings_matrix)
index.add(embeddings_matrix)

# Função de extração de categorias
def extrair_categorias(query):
    doc = nlp(query.lower())
    categorias = []
    categoria_atual = []

    stopwords = {"difference", "comparison", "know", "want", "between", "and", "or", "vs", "to", "the", "which", "what"}

    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"] and token.text not in stopwords:
            categoria_atual.append(token.text)
        elif token.pos_ == "ADJ":
            categoria_atual.append(token.text)
        else:
            if categoria_atual:
                categorias.append(" ".join(categoria_atual))
                categoria_atual = []
    if categoria_atual:
        categorias.append(" ".join(categoria_atual))

    if len(categorias) > 2:
        categorias = categorias[-2:]

    return categorias

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
# transforma o que o usuário digita em uma ação inteligente e focada: comparar categorias ou analisar produtos por período.
import spacy
import faiss
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
from nltk.corpus import wordnet
from sklearn.preprocessing import MinMaxScaler

# Carregar o modelo spaCy
nlp = spacy.load("en_core_web_sm")

def  detecta_query_tipo(query):
    periodos = extrair_periodos(query)
    categorias = extrair_categorias(query)

    if periodos and categorias:
        return "period_analysis"  # Produto + Período
    elif len(categorias) >= 2:
        return "category_comparison"  # Duas categorias
    else:
        return "unknown"

# Função para extrair categorias usando o dicionário
def extrair_categorias(query):
    doc = nlp(query.lower())
    extracted = []
    stopwords = {"difference", "comparison", "know", "want", "between", "and", "or", "vs", "to", "the", "which", "what"}

    # Detecta frases nominais como "home entertainment"
    frases_sinon = [chunk.text.strip().lower() for chunk in doc.noun_chunks]

    for frase in frases_sinon:
        frase = frase.strip()
        if frase in categorias:
            extracted.append(frase)
        else:
            for syn_list in categorias_sinonimos.values():
                if frase in syn_list:
                    extracted.append(frase)

    return list(set(extracted))

# Função para extrair períodos
def extrair_periodos(query):
    periodos_conhecidos = {
        "summer", "winter", "spring", "autumn",
        "january", "february", "march", "april", "may", "june",
        "july", "august", "september", "october", "november", "december",
        "new ano", "valentines dia", "mothers dia", "black fridia", "christmas"
    }

    query = query.lower()
    doc = nlp(query)
    frases = [chunk.text.lower() for chunk in doc.noun_chunks]
    tokens = [token.text.lower() for token in doc]

    detected = []

    for frase in frases + tokens:
        if frase in periodos_conhecidos and frase not in detected:
            detected.append(frase)

    # Verificação manual para datas compostas (caso o spaCy falhe)
    for period in periodos_conhecidos:
        if " " in period and period in query and period not in detected:
            detected.append(period)

    return detected

def expandir_query_com_sinonimos(query, categorias_sinonimos, context_words=None):
    words = query.lower().split()
    expanded_words = []

    for word in words:
        expanded_words.append(word)
        for category, synonyms in categorias_sinonimos.items():
            if word in synonyms or word == category:
                expanded_words.extend(synonyms)

    if context_words:
        expanded_words.extend(context_words)

    return " ".join(list(set(expanded_words)))

# Buscar produtos no índice FAISS
def query_index(query_text, max_resultados=50, distance_threshold=2, query_tipo="category_comparison"):
    # Definir palavras de contexto
    if query_tipo == "period_analysis":
        context_words = ["sale", "sold", "discount", "estacao", "best seller"]
    else:
        context_words = ["compare", "difference", "specs", "features"]

    expanded_query = expandir_query_com_sinonimos(query_text, categorias_sinonimos, context_words)

    query_embedding = embedding_model.encode([expanded_query], normalize_embeddings=True)
    distances, indices = index.search(query_embedding, max_resultados)

    resultados = df.iloc[indices[0]].copy()
    resultados["distance"] = distances[0]
    resultados_filtrados = resultados[resultados["distance"] <= distance_threshold]

    if resultados_filtrados.empty:
        print("Nenhum produto encontrado com distância baixa, retornando os melhores disponíveis.")
        resultados_filtrados = resultados.head(10)

    if not resultados_filtrados.empty:
        resultados_filtrados = resultados_filtrados.sort_values(by="sales_quantity", ascending=False)

    return resultados_filtrados.reset_index(drop=True)

# Buscar produtos por período
def query_index_by_period(query_text, period_mess, max_resultados=50, distance_threshold=2):
    df['data'] = pd.to_datatime(df['data'], errors='coerce')
    df['mes'] = df['data'].dt.mes
    period_df = df[df['mes'].isin(period_mess)]

    if period_df.empty:
        print(f"Nenhum dado encontrado para o período {period_mess}.")
        return pd.DataFrame()

    # Expandir a consulta para venda
    expanded_query = expandir_query_com_sinonimos(query_text, categorias_sinonimos, context_words=["sale", "sold", "discount", "estacao", "best seller"])

    query_embedding = embedding_model.encode([expanded_query], normalize_embeddings=True)
    distances, indices = index.search(query_embedding, max_resultados)

    valid_indices = [i for i in indices[0] if i < len(period_df)]

    if not valid_indices:
        print(f"Nenhum resultado FAISS válido para o período {period_mess}.")
        return pd.DataFrame()

    resultados = period_df.iloc[valid_indices].copy()
    resultados["distance"] = distances[0][:len(valid_indices)]

    resultados_filtrados = resultados[resultados["distance"] <= distance_threshold]

    if resultados_filtrados.empty:
        print("Nenhum produto encontrado com distância baixa no período, retornando os melhores disponíveis.")
        resultados_filtrados = resultados.head(10)

    if not resultados_filtrados.empty:
        resultados_filtrados = resultados_filtrados.sort_values(by="sales_quantity", ascending=False)

    return resultados_filtrados.reset_index(drop=True)

# Gerar resposta por categoria
def generate_response(query):
    categorias = extrair_categorias(query)
    if isinstance(categorias, str):
        return categorias

    response_list = {}

    for category in categorias:
        print(f"Buscando produtos relacionados a: {category}")
        resultados = query_index(category, max_resultados=50)

        response_list[category.capitalize()] = []

        for _, row in resultados.iterrows():
            response_list[category.capitalize()].append({
                "product_id": row["product_id"],
                "product_name": row["product_name"],
                "product_link": row["product_link"]
            })

        if not response_list[category.capitalize()]:
            response_list[category.capitalize()].append({
                "product_id": None,
                "product_name": f"Nenhum produto encontrado relacionado a '{category.capitalize()}'.",
                "product_link": None
            })

    print("\n--- Resultados por Categoria ---")
    for category, products in response_list.items():
        print(f"\nCategoria: {category}")
        for product in products:
            if isinstance(product, dict):
                print(f"- {product['product_name']} (ID: {product['product_id']}) - {product['product_link']}")
            else:
                print(product)

    return response_list

# Dados estruturados que serão usados no gráfico e no texto via RAG
total_sales = 0;
category_total_sales = {}  # Ex: {"Phone": 1200}

# Gerar resposta considerando períodos
def generate_period_response(query):
    periodos = {
        "summer": [6, 7, 8],
        "winter": [12, 1, 2],
        "spring": [3, 4, 5],
        "autumn": [9, 10, 11],
        "january": [1],
        "february": [2],
        "march": [3],
        "april": [4],
        "may": [5],
        "june": [6],
        "july": [7],
        "august": [8],
        "september": [9],
        "october": [10],
        "november": [11],
        "december": [12]
    }

    # Datas especiais tratadas como períodos nomeados
    data_especial = {
      "new ano": "01-01",
      "valentines dia": "06-12",
      "mothers dia": "05-12",
      "black fridia": "11-24",
      "christmas": "12-25"
       }

    detected_periodos = extrair_periodos(query)
    print(f"Periodos detectados: {detected_periodos}")

    categorias = extrair_categorias(query)
    print(f"Categorias detectadas: {categorias}")

    if not detected_periodos:
        print("Nenhum período detectado na consulta.")
        return {}
    if not categorias:
        print("Nenhuma categoria detectada na consulta.")
        return {}

    response_list = {}

    global category_total_sales
    global total_sales

    for category in categorias:
        print(f"\nBuscando produtos para a categoria: {category.capitalize()}")
        category_resultados = query_index(category, max_resultados=50)

        if category_resultados.empty:
            # Corrigir aqui: colocar dict vazio, mas formato correto
            response_list[category.capitalize()] = {"Geral": [{
                "product_id": None,
                "product_name": f"Nenhum produto encontrado para a categoria '{category.capitalize()}'.",
                "product_link": None
            }]}
        else:
            category_resultados['date'] = pd.to_datetime(category_resultados['date'], errors='coerce')
            category_resultados['mes'] = category_resultados['date'].dt.month
            print(f"Meses extraídos: {category_resultados['mes'].unique()}")
            category_resultados['sales_quantity'] = pd.to_numeric(category_resultados['sales_quantity'], errors='coerce')
            total_sales = category_resultados['sales_quantity'].sum()
            category_total_sales[category.capitalize()] = total_sales


            filtered_by_periodos = {}
            #######
            for period in detected_periodos:
                print(f"\nFiltrando produtos para o período: {period.capitalize()}")

                if period in data_especial:
                    # Tratar como uma data exata
                    mes_str, dia_str = data_especial[period].split("-")
                    evento_mes = int(mes_str)
                    evento_dia = int(dia_str)

                    category_resultados["dia"] = category_resultados["data"].dt.dia
                    category_resultados["mes"] = category_resultados["data"].dt.mes
                    # +/- 2 dias de diferença da data
                    period_resultados = category_resultados[
                        (category_resultados["mes"] == evento_mes) &
                        (abs(category_resultados["dia"] - evento_dia) <= 2)
                    ]
                elif period in periodos:
                    print(f"Meses disponíveis para filtro: {periodos[period]}")
                    period_resultados = category_resultados[category_resultados["mes"].isin(periodos[period])]
                else:
                    print(f"Período não reconhecido: {period}")
                    continue

                print(f"Filtrados: {period_resultados.shape[0]} produtos para o período {period.capitalize()}")

                if period_resultados.empty:
                    filtered_by_periodos[period.capitalize()] = [{
                        "product_id": None,
                        "product_name": f"Nenhum produto de '{category.capitalize()}' encontrado para o período '{period.capitalize()}'.",
                        "product_link": None
                    }]
                else:
                    filtered_by_periodos[period.capitalize()] = []
                    for _, row in period_resultados.iterrows():
                        filtered_by_periodos[period.capitalize()].append({
                            "product_id": row["product_id"],
                            "product_name": row["product_name"],
                            "product_link": row["product_link"]
                        })

            if filtered_by_periodos:
                response_list[category.capitalize()] = filtered_by_periodos

    print("\n--- Resultados por Período ---")
    for category, period_resultados in response_list.items():
        print(f"\nCategoria: {category}")
        for period, products in period_resultados.items():
            print(f"\nPeríodo: {period}")
            for product in products:
                if isinstance(product, dict):
                    print(f"- {product['product_id']} - {product['product_name']} - {product['product_link']}")
                else:
                    print(product)

    return response_list

# tv camera sold in spring and winter
# difference in sales between phone and tv
query = "tv camera sold in spring and winter"

query_tipo =  detecta_query_tipo(query)

if query_tipo == "period_analysis":
    print("\n Detectado: Análise de Vendas por Período")
    global period_resultados
    period_resultados = generate_period_response(query)

elif query_tipo == "category_comparison":
    print("\n Detectado: Comparação de Categorias")
    global resultados
    resultados = generate_response(query)
else:

    print("\n Não foi possível identificar o tipo de consulta.")
print(total_sales)


 Detectado: Análise de Vendas por Período
Periodos detectados: ['spring', 'winter']
Categorias detectadas: ['tv camera']

Buscando produtos para a categoria: Tv camera
Meses extraídos: [ 4  5  6  9  3  7 12 11  2 10  8  1]

Filtrando produtos para o período: Spring
Meses disponíveis para filtro: [3, 4, 5]
Filtrados: 12 produtos para o período Spring

Filtrando produtos para o período: Winter
Meses disponíveis para filtro: [12, 1, 2]
Filtrados: 11 produtos para o período Winter

--- Resultados por Período ---

Categoria: Tv camera

Período: Spring
- B014I8SSD0 - Amazon Basics High-Speed HDMI Cable, 6 Feet - Supports Ethernet, 3D, 4K video,Black - https://www.amazon.in/AmazonBasics-High-Speed-HDMI-Cable-Feet/dp/B014I8SSD0/ref=sr_1_51?qid=1672909126&s=electronics&sr=1-51
- B07YY1BY5B - Noise ColorFit Pro 2 Full Touch Control Smart Watch with 35g Weight & Upgraded LCD Display,IP68 Waterproof,Heart Rate Monitor,Sleep & Step Tracker,Call & Message Alerts & Long Battery Life (Jet Black) - ht

In [47]:
# Essa célula prepara o motor de busca que o sistema vai usar para encontrar produtos parecidos sem depender de palavras exatas.

# Gerar os embeddings para cada produto
embeddings = np.array([embedding_model.encode(str(text)) for text in df['product_name']])

# Certificar-se de que os embeddings estão no formato 2D (n, d)
embeddings = embeddings.reshape(len(df), -1)

# Criar o índice FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])  # Usando uma indexação simples

# Adicionar os embeddings ao índice
index.add(embeddings.astype(np.float32))  # FAISS exige embeddings como float32

# Salvar o índice em um arquivo
faiss.write_index(index, "index")

In [41]:
# Análise comparativa por categorias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import faiss
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.preprocessing import MinMaxScaler
from matplotlib.patches import Patch
from math import pi
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Carregar modelos
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
t5_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Função para buscar produtos no FAISS
def search_faiss(query, faiss_index, df, top_k=10):
    query_embedding = embedding_model.encode([query], normalize_embeddings=True)
    _, indices = faiss_index.search(query_embedding, top_k)
    return df.iloc[indices[0]].dropna()

# Função RAG
def generate_comparative_rag_text(df_metrics, tokenizer, t5_model):
    context = ""
    for _, row in df_metrics.iterrows():
        context += (
            f"Category: {row['Categoria']}, "
            f"Total Sales: {row['Total de Vendas']}, "
            f"Revenue: {row['Receita Estimada']:.2f}, "
            f"Average Ticket: {row['Ticket Médio']:.2f}, "
            f"Average Discount: {row['Desconto Médio (%)']:.2f}%, "
            f"Average Rating: {row['Avaliação Média']:.2f}. "
        )

    prompt = (
        "You are a data analyst. Based on the following product category sales data, "
        "perform a comparative analysis highlighting which category performs better in total sales, revenue, discount, and user rating. "
        "Then, provide actionable business insights:\n" + context
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = t5_model.generate(**inputs, max_length=250)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Função principal
def generate_charts_and_rag_text(resultados, df, faiss_index):
    categorias = list(resultados.keys())
    metrics = []

    total_sales_all = df["sales_quantity"].sum()
    print(f"Total geral de vendas (todos os produtos): {total_sales_all}")

    for category in categorias:
        sales = 0
        revenue = 0
        discounts = []
        ratings = []
        for product in resultados[category]:
            prod_data = df[df['product_id'] == product['product_id']]
            if not prod_data.empty:
                qty = prod_data["sales_quantity"].values[0]
                price = prod_data["discounted_price"].values[0]
                disc = prod_data["discount_percentage"].values[0]
                rate = prod_data["rating"].values[0]
                sales += qty
                revenue += qty * price
                discounts.append(disc)
                ratings.append(rate)

        ticket_medio = revenue / sales if sales else 0
        avg_discount = np.mean(discounts) if discounts else 0
        avg_rating = np.mean(ratings) if ratings else 0

        metrics.append({
            "Categoria": category,
            "Total de Vendas": sales,
            "Receita Estimada": revenue,
            "Ticket Médio": ticket_medio,
            "Desconto Médio (%)": avg_discount,
            "Avaliação Média": avg_rating
        })

    df_metrics = pd.DataFrame(metrics)

    # Pizza
    pie_labels = df_metrics["Categoria"].tolist()
    pie_values = df_metrics["Total de Vendas"].tolist()
    others = total_sales_all - sum(pie_values)
    if others > 0:
        pie_labels.append("Others")
        pie_values.append(others)

    cores_marcadores = ["rgb(179, 205, 227)", "rgb(197,222,105)", "#1c8356"][:len(pie_labels)]

    # Normalizar dados para gráfico de barras (range 50–80)
    cols_to_normalize = ['Total de Vendas', 'Receita Estimada', 'Ticket Médio', 'Desconto Médio (%)', 'Avaliação Média']
    scaler = MinMaxScaler(feature_range=(50, 80))
    values_scaled = scaler.fit_transform(df_metrics[cols_to_normalize])
    df_norm = pd.DataFrame(values_scaled, columns=cols_to_normalize)
    df_norm["Categoria"] = df_metrics["Categoria"]

    # Criar subplot com pizza e barras
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "domain"}, {"type": "xy"}]], column_widths=[0.4, 0.6])

    # Pizza
    fig.add_trace(go.Pie(
        labels=pie_labels,
        values=pie_values,
        marker_colors=cores_marcadores,
        hole=0.5,
        pull=[0.1 if label == "Others" else 0 for label in pie_labels],
        textposition="outside",
        textinfo="percent+label"
    ), row=1, col=1)

    # Barras (normalizadas)
    df_transposta = df_norm.set_index("Categoria")[cols_to_normalize].T.reset_index()
    df_transposta = df_transposta.rename(columns={"index": "Indicador"})

    cores_barras = {
        df_norm["Categoria"].iloc[0]: "rgb(179, 205, 227)",  # Azul claro
        df_norm["Categoria"].iloc[1]: "rgb(197, 222, 105)"   # Verde amarelado
    }

    bordas_barras = {
        df_norm["Categoria"].iloc[0]: "rgb(100, 150, 200)",  # Borda azul escuro
        df_norm["Categoria"].iloc[1]: "rgb(130, 170, 70)"    # Borda verde escuro
    }

    for categoria in df_norm["Categoria"]:
        fig.add_trace(go.Bar(
            x=df_transposta["Indicador"],
            y=df_transposta[categoria],
            name=categoria,
            marker=dict(
                color=cores_barras.get(categoria, "#cccccc"),
                line=dict(color=bordas_barras.get(categoria, "#444444"), width=2)
            )
        ), row=1, col=2)

    # Ajustes finais do layout
    fig.update_layout(
        title=None,
        barmode='group',
        template='ggplot2',
        font=dict(family='Arial', size=14),
        plot_bgcolor='rgb(248, 248, 255)',
        paper_bgcolor='rgb(248, 248, 255)',
        margin=dict(l=80, r=40, t=80, b=80),
        xaxis2=dict(
            title="Indicador",
            tickangle=0,
            showgrid=True,
            zeroline=False
        ),
        yaxis2=dict(
            title="Valor Normalizado (0 a 100)",
            tickmode='array',
            tickvals=[0, 25, 50, 75, 100],
            showgrid=True,
            gridcolor='rgba(200,200,200,0.3)'
        ),
        legend=dict(
            title=dict(text="Categorias", font=dict(size=14, color="#333333")),
            orientation="h",
            x=0,
            y=-0.25,
            xanchor="left",
            font=dict(size=12, color="#333333")
        ),
        annotations=[
            dict(
                text="Análise Comparativa por Categoria",
                xref="paper", yref="paper",
                x=0, y=1.15, showarrow=False,
                font=dict(size=20, color="#333333")
            ),
            dict(
                text="Vendas",
                xref="paper", yref="paper",
                x=0.13, y=0.5, showarrow=False,
                font=dict(size=20, color="#333333")
            )
        ]
    )
    fig.show()

    # Mostrar valores reais usados nas métricas
    print("\n--- Valores reais usados no gráfico de barras ---")
    print(df_metrics)

    # Texto gerado por IA
    generated_text = generate_comparative_rag_text(df_metrics, tokenizer, t5_model)
    print("\n- Insights (com RAG):\n")
    print(generated_text)

# Carregar dados e índice
df = pd.read_csv("amazon_with_sales.csv")
faiss_index = faiss.read_index("index")

# Chamada da função (certifique-se de que a variável 'resultados' esteja definida antes)
generate_charts_and_rag_text(resultados, df, faiss_index)

Total geral de vendas (todos os produtos): 148967



--- Valores reais usados no gráfico de barras ---
  Categoria  Total de Vendas  Receita Estimada  Ticket Médio  \
0        Tv            14875        27860683.0   1872.987092   
1     Phone            11605        10597671.0    913.198707   

   Desconto Médio (%)  Avaliação Média  
0               48.16            4.204  
1               64.08            3.898  

- Insights (com RAG):

Total sales for the category are 14875, revenue is 27860683.00, discount is 48.16%, average rating is 4.20.


In [65]:
# Análise comparativa por períodos
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.preprocessing import MinMaxScaler

# Carregar modelos
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
t5_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Função para gerar texto RAG comparativo por período
def generate_comparative_rag_text_period(df_metrics, total_category_sales, tokenizer, t5_model):
    context = ""
    for _, row in df_metrics.iterrows():
        context += (
            f"Period: {row['Período']}, "
            f"Sales in Period: {row['Total de Vendas']}, "
            f"Share of Total: {row['% no Total da Categoria']:.2f}%, "
            f"Revenue: {row['Receita Estimada']:.2f}, "
            f"Avg Ticket: {row['Ticket Médio']:.2f}, "
            f"Avg Discount: {row['Desconto Médio (%)']:.2f}%, "
            f"Avg Rating: {row['Avaliação Média']:.2f}. "
        )

    prompt = (
        "You are a senior sales analyst. Given the sales data below, compare the periodos in terms of total sales, revenue, discount, and rating."
        "perform a comparative analysis highlighting which category performs better in total sales, revenue, discount, and user rating."
        "Then, provide actionable business insights:\n" + context
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = t5_model.generate(**inputs, max_length=400)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# função principal
def generate_charts_and_rag_text(resultados, df, category_total_sales):
    period_list = list(resultados[list(resultados.keys())[0]].keys())
    category_name = list(resultados.keys())[0]
    total_sales = category_total_sales.get(category_name, 0)

    metrics = []
    for period in period_list:
        sales = 0
        revenue = 0
        discounts = []
        ratings = []

        for product in resultados[category_name][period]:
            product_id = str(product['product_id']).strip()
            product_info = df[df['product_id'].astype(str).str.strip() == product_id]
            if product_info.empty:
                continue

            qty = pd.to_numeric(product_info["sales_quantity"].values[0], errors='coerce')
            price = pd.to_numeric(product_info["discounted_price"].values[0], errors='coerce')
            disc = pd.to_numeric(product_info["discount_percentage"].values[0], errors='coerce')
            rate = pd.to_numeric(product_info["rating"].values[0], errors='coerce')

            if pd.isna(qty) or pd.isna(price):
                continue

            sales += qty
            revenue += qty * price
            discounts.append(disc)
            ratings.append(rate)

        ticket_medio = revenue / sales if sales else 0
        avg_discount = np.mean(discounts) if discounts else 0
        avg_rating = np.mean(ratings) if ratings else 0
        pct_total = (sales / total_sales * 100) if total_sales else 0

        metrics.append({
            "Período": period,
            "Total de Vendas": sales,
            "Receita Estimada": revenue,
            "Ticket Médio": ticket_medio,
            "Desconto Médio (%)": avg_discount,
            "Avaliação Média": avg_rating,
            "% no Total da Categoria": pct_total
        })

    df_metrics = pd.DataFrame(metrics)

    # Cores e bordas
    cores_barras = {
        df_metrics["Período"].iloc[0]: "rgb(179, 205, 227)",
        df_metrics["Período"].iloc[1]: "rgb(197, 222, 105)"
    }
    bordas_barras = {
        df_metrics["Período"].iloc[0]: "rgb(100, 150, 200)",
        df_metrics["Período"].iloc[1]: "rgb(130, 170, 70)"
    }

    # Normalizar para gráfico de barras
    cols_to_normalize = ['Total de Vendas', 'Receita Estimada', 'Ticket Médio', 'Desconto Médio (%)', 'Avaliação Média']
    scaler = MinMaxScaler(feature_range=(50, 80))
    values_scaled = scaler.fit_transform(df_metrics[cols_to_normalize])
    df_norm = pd.DataFrame(values_scaled, columns=cols_to_normalize)
    df_norm["Período"] = df_metrics["Período"]

    # Gráfico de Pizza + Barras
    pie_labels = df_metrics["Período"].tolist()
    pie_values = df_metrics["% no Total da Categoria"].tolist()

    outros_pct = 100 - sum(pie_values)
    if outros_pct > 0.5:
        pie_labels.append("Others")
        pie_values.append(outros_pct)
        cores_barras["Others"] = "#1c8356"
        bordas_barras["Others"] = "#0d3a23"

    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "domain"}, {"type": "xy"}]], column_widths=[0.4, 0.6])

    fig.add_trace(go.Pie(
        labels=pie_labels,
        values=pie_values,
        marker=dict(
            colors=[cores_barras[p] for p in pie_labels]
        ),
        hole=0.5,
        pull=[0.1 if p == "Others" else 0 for p in pie_labels],
        textposition="outside",
        textinfo="percent+label"
    ), row=1, col=1)

    df_transposta = df_norm.set_index("Período")[cols_to_normalize].T.reset_index().rename(columns={"index": "Indicador"})

    for periodo in df_norm["Período"]:
        fig.add_trace(go.Bar(
            x=df_transposta["Indicador"],
            y=df_transposta[periodo],
            name=periodo,
            marker=dict(
                color=cores_barras[periodo],
                line=dict(color=bordas_barras[periodo], width=2)
            )
        ), row=1, col=2)

    fig.update_layout(
        barmode='group',
        title=None,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        xaxis2=dict(
            title="Indicador",
            showgrid=False,
            zeroline=False,
            showline=False,
            showticklabels=True
        ),
        yaxis2=dict(
            title="Valor Normalizado (0 a 100)",
            showgrid=False,
            zeroline=False,
            showline=False,
            showticklabels=True,
            dtick=25  # Intervalos de 25 no eixo y
        ),
        legend=dict(
            title=dict(text="Períodos"),
            orientation="h",
            xanchor="left",
            x=0,
            y=-0.25
        ),
        annotations=[
            dict(
                text="Vendas",
                xref="paper", yref="paper",
                x=0.15, y=0.5,
                showarrow=False,
                font=dict(size=20, color="rgb(50, 50, 50)")
            ),
            dict(
                text=f"Análise por Período - Categoria '{category_name}'",
                xref="paper", yref="paper",
                x=0, y=1.1,
                showarrow=False,
                font=dict(size=18, color="#333333")
            )
        ]
    )

    fig.show()

    print("\n--- Valores reais usados nas métricas ---")
    print(df_metrics)

    # RAG
    generated_text = generate_comparative_rag_text_period(df_metrics, total_sales, tokenizer, t5_model)
    print("\n- Insights (com RAG):\n")
    print(generated_text)

generate_charts_and_rag_text(period_resultados, df, category_total_sales)


--- Valores reais usados nas métricas ---
  Período  Total de Vendas  Receita Estimada  Ticket Médio  \
0  Spring             2655        6001792.24   2260.562049   
1  Winter             1003        1817607.00   1812.170489   

   Desconto Médio (%)  Avaliação Média  % no Total da Categoria  
0           59.500000         4.100000                38.075434  
1           68.090909         4.081818                14.384053  

- Insights (com RAG):

Total sales for the period was 2655 compared to 1003 in terms of revenue and discount. The average ticket price was 1817607.00 and the average discount was 68.09%.
